# Project 1 Data scraping and api
## BAIS:3250 -- Data Wrangling
### Connor Ellis

In [2]:
# Import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Function to scrape COVID-19 data from Worldometer using Selenium with Chrome
def scrape_worldometer_covid_data():
    """
    Scrapes COVID-19 daily case and death data from Worldometer's US page
    using Selenium with Chrome browser.
    Returns a pandas DataFrame with dates, new cases, and new deaths.
    """
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (no browser UI)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    # Navigate to the Worldometer coronavirus US page
    url = "https://www.worldometers.info/coronavirus/country/us/"
    print(f"Accessing {url}...")
    driver.get(url)
    
    # Wait for the page to load completely (especially the charts)
    time.sleep(5)
    
    # Get the page source after JavaScript has executed
    page_source = driver.page_source
    
    # Close the driver
    driver.quit()
    
    # Parse the HTML content
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find all scripts in the page
    scripts = soup.find_all('script')
    
    # Initialize variables to store our data
    dates = []
    new_cases = []
    new_deaths = []
    
    print("Extracting data from scripts...")
    
    # Search through scripts for the daily cases and deaths data
    for script in scripts:
        if script.string and 'Highcharts.chart' in script.string:
            script_content = script.string
            
            # Extract daily new cases data
            if 'graph-cases-daily' in script_content:
                print("Found daily cases graph data")
                # Extract categories (dates)
                categories_match = re.search(r'categories: \[(.*?)\]', script_content, re.DOTALL)
                if categories_match:
                    categories_str = '[' + categories_match.group(1) + ']'
                    # Clean up the dates string and convert to list
                    dates = eval(categories_str.replace("'", "\""))
                    print(f"Extracted {len(dates)} dates")
                
                # Extract data (cases)
                data_match = re.search(r'data: \[(.*?)\]', script_content, re.DOTALL)
                if data_match:
                    data_str = '[' + data_match.group(1) + ']'
                    # Convert to integers, handling null values
                    cases_data = eval(data_str.replace("null", "None"))
                    new_cases = [0 if x is None else x for x in cases_data]
                    print(f"Extracted {len(new_cases)} case data points")
            
            # Extract daily new deaths data
            if 'graph-deaths-daily' in script_content:
                print("Found daily deaths graph data")
                # Extract data (deaths)
                data_match = re.search(r'data: \[(.*?)\]', script_content, re.DOTALL)
                if data_match:
                    data_str = '[' + data_match.group(1) + ']'
                    # Convert to integers, handling null values
                    deaths_data = eval(data_str.replace("null", "None"))
                    new_deaths = [0 if x is None else x for x in deaths_data]
                    print(f"Extracted {len(new_deaths)} death data points")
    
    # Check if we have data
    if not dates or not new_cases:
        print("Failed to extract COVID-19 data")
        return None
    
    # Ensure all lists have the same length (use the shortest)
    min_length = min(len(dates), len(new_cases), len(new_deaths) if new_deaths else len(new_cases))
    dates = dates[:min_length]
    new_cases = new_cases[:min_length]
    new_deaths = new_deaths[:min_length] if new_deaths else [0] * min_length
    
    # Convert string dates to datetime objects
    try:
        date_objects = [datetime.strptime(date, "%b %d, %Y") for date in dates]
    except ValueError:
        # If the format is different, try an alternative format
        date_objects = [datetime.strptime(date, "%B %d, %Y") for date in dates]
    
    # Create a DataFrame
    covid_df = pd.DataFrame({
        'Date': date_objects,
        'New_Cases': new_cases,
        'New_Deaths': new_deaths
    })
    
    # Set Date as index
    covid_df.set_index('Date', inplace=True)
    
    print(f"Successfully created DataFrame with {len(covid_df)} rows")
    
   
    
    return covid_df

# Function to save the scraped data to a CSV file


# Example usage
if __name__ == "__main__":
    # To run in a Jupyter notebook, remove this if block and execute the following:
    covid_data = scrape_worldometer_covid_data()
    
    if covid_data is not None:
        # Display the first few rows
        display(covid_data.head())
        
        
        # Show some basic statistics
        print("\nBasic Statistics:")
        display(covid_data.describe())

Accessing https://www.worldometers.info/coronavirus/country/us/...
Extracting data from scripts...
Found daily cases graph data
Extracted 1519 dates
Extracted 1519 case data points
Found daily deaths graph data
Extracted 1519 death data points
Successfully created DataFrame with 1519 rows


,New_Cases,New_Deaths
Date,,
2020-02-15,0,0
2020-02-16,0,0
2020-02-17,0,0
2020-02-18,0,0
2020-02-19,0,0



Basic Statistics:


,New_Cases,New_Deaths
count,1519.000000,1519.000000
mean,73614.263989,802.822251
std,107565.144145,847.995975
min,0.000000,0.000000
25%,19489.500000,191.500000
50%,43153.000000,463.000000
75%,85852.500000,1117.000000
max,914412.000000,4402.000000


In [3]:
#check datatypes
covid_data.dtypes

New_Cases     int64
New_Deaths    int64
dtype: object

In [4]:
#import second data source
SP500df = pd.read_csv('SP500.csv', encoding = 'utf-8', sep = ',')
#display
SP500df

,DATE,VALUE
0,2015-03-30,2086.24
1,2015-03-31,2067.89
2,2015-04-01,2059.69
3,2015-04-02,2066.96
4,2015-04-03,.
...,...,...
2605,2025-03-24,5767.57
2606,2025-03-25,5776.65
2607,2025-03-26,5712.20
2608,2025-03-27,5693.31


In [5]:
# Rename columns to lowercase
SP500df = SP500df.rename(columns={'DATE': 'Date', 'VALUE': 'Value'})

#set date to index
SP500df.set_index('Date', inplace = True)
SP500df


,Value
Date,
2015-03-30,2086.24
2015-03-31,2067.89
2015-04-01,2059.69
2015-04-02,2066.96
2015-04-03,.
...,...
2025-03-24,5767.57
2025-03-25,5776.65
2025-03-26,5712.20


In [6]:
#check for missing values in both
print(SP500df.isna().sum())
print(covid_data.isna().sum())
#change value to float

SP500df.dtypes

Value    0
dtype: int64
New_Cases     0
New_Deaths    0
dtype: int64


Value    object
dtype: object

In [7]:
SP500df['Value'] = pd.to_numeric(SP500df['Value'], errors='coerce') #change to numeric
#change date to datetime for merge
SP500df.index = pd.to_datetime(SP500df.index)
SP500df.dtypes

Value    float64
dtype: object

In [8]:
#horizontally integrate data
project_data = pd.merge(covid_data, SP500df, on = 'Date', how = 'inner')
project_data

,New_Cases,New_Deaths,Value
Date,,,
2020-02-17,0,0,NaN
2020-02-18,0,0,3370.29
2020-02-19,0,0,3386.15
2020-02-20,0,0,3373.23
2020-02-21,20,0,3337.75
...,...,...,...
2024-04-08,2331,28,5202.39
2024-04-09,2361,38,5209.91
2024-04-10,2555,30,5160.64


In [9]:
#check for missing values
project_data.isna().sum()

New_Cases      0
New_Deaths     0
Value         39
dtype: int64

In [10]:
#save integrated data to csv
project_data.to_csv('cellis3_project_data.csv', encoding = 'utf-8')

In [59]:
#add 3rd data set
mobilitydf = pd.read_csv('2020_US_Region_Mobility_Report.csv', encoding = 'utf-8')
mobilitydf = mobilitydf[mobilitydf['sub_region_1'].isna()]
mobilitydf = mobilitydf.drop(columns = ['sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code','place_id'])
mobilitydf = mobilitydf.rename(columns = {'date':'Date'})
mobilitydf

,country_region_code,country_region,Date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,US,United States,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,US,United States,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,US,United States,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,US,United States,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
316,US,United States,2020-12-27,-27.0,-18.0,-12.0,-31.0,-18.0,7.0
317,US,United States,2020-12-28,-17.0,-9.0,-12.0,-40.0,-44.0,14.0
318,US,United States,2020-12-29,-17.0,-6.0,-8.0,-39.0,-45.0,14.0
319,US,United States,2020-12-30,-13.0,0.0,-11.0,-37.0,-44.0,14.0


In [77]:
projectdf = pd.read_csv('cellis3_project_data.csv', encoding = 'utf-8')
projectdata = pd.merge(projectdf, mobilitydf, on = 'Date', how = 'left')
projectdata.to_csv('cellis3_project_data.csv', encoding = 'utf-8')

AttributeError: 'numpy.int64' object has no attribute 'head'